<a href="https://colab.research.google.com/github/DeepsMoseli/TswanaBert/blob/master/tswanaBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install transformers

In [23]:

from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path("./").glob("**/*.txt")]
paths

['large_test_clean.txt',
 'drive/My Drive/2017 Applications/datasets/classification/bank/bank-names.txt',
 'drive/My Drive/2017 Applications/datasets/classification/bank/bank-additional/bank-additional-names.txt',
 'drive/My Drive/2017 Applications/datasets/classification/skin colour/Skin_NonSkin.txt',
 'drive/My Drive/2017 Applications/datasets/classification/student/student.txt',
 'drive/My Drive/dissertation/MIT807/latex tems/Dissertate-NYU-LaTeX/fonts/Lato/SIL Open Font License 1.1.txt',
 'drive/My Drive/dissertation/MIT807/Code/crowdAI.txt',
 'drive/My Drive/dissertation/MIT807/backup/changelog.txt',
 'drive/My Drive/dissertation/MIT807/backup/backup/changelog.txt',
 'drive/My Drive/dissertation/MIT807/backup/accuracy_crossValidation/changelog.txt',
 'drive/My Drive/Valuable code/aloc price formula.txt',
 'drive/My Drive/Valuable code/AR EndImportPortfolio.txt',
 'drive/My Drive/Valuable code/Alloc Enhencements.txt',
 'drive/My Drive/Valuable code/testAggregation.txt',
 'drive/My 

## initialize tokenizer and customize training


In [25]:
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=["./large_test_clean.txt"], show_progress=True,vocab_size=25000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])
tokenizer.save_model('./drive/My Drive/TswanaBert')
print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

Trained vocab size: 23372


In [26]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./drive/My Drive/TswanaBert/vocab.json",
    "./drive/My Drive/TswanaBert/merges.txt",
)

tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)


encoding = tokenizer.encode("gore fa a re merafe yotlhe e nne le.")
print("Encoded String: ", encoding.tokens)

decoded = tokenizer.decode(encoding.ids)
print("Decoded string: {}".format(decoded))

Encoded String:  ['<s>', 'gore', 'Ġfa', 'Ġa', 'Ġre', 'Ġmerafe', 'Ġyotlhe', 'Ġe', 'Ġnne', 'Ġle', '.', '</s>']
Decoded string: <s>gore fa a re merafe yotlhe e nne le.</s>


## Make and Train a language model



In [27]:
!nvidia-smi

Sat Jul  4 02:53:00 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    25W /  75W |   7261MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
torch.cuda.is_available()

False

### Config the model and re-create our tokenizer in transformers


In [17]:
from transformers import RobertaConfig
from transformers import RobertaTokenizerFast

config = RobertaConfig(
    vocab_size = tokenizer.get_vocab_size(),
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

tokenizer = RobertaTokenizerFast.from_pretrained("./drive/My Drive/TswanaBert/", max_len=512)

 since we are training from scratch, we initialize from config not a checkpoint of saved model


In [18]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config = config)
print("Number of parameters: ", model.num_parameters())

Number of parameters:  62080076


## Make and register training dataset

In [19]:
%%time
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./large_test_clean.txt",
    block_size=128,
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

CPU times: user 1.52 s, sys: 29.8 ms, total: 1.55 s
Wall time: 912 ms


Initialize model

In [20]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./drive/My Drive/TswanaBert",
    overwrite_output_dir=True,
    num_train_epochs=250,
    per_device_train_batch_size=64,
    save_steps=15_000,
    save_total_limit=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [21]:
%%time
trainer.train()

RuntimeError: ignored

In [ ]:
trainer.save_model("./TswanaBert")
tokenizer.save_pretrained("./TswanaBert")

('./TswanaBert/vocab.json',
 './TswanaBert/merges.txt',
 './TswanaBert/special_tokens_map.json',
 './TswanaBert/added_tokens.json')

-----
# Now lets Test

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./TswanaBert",
    tokenizer="./TswanaBert"
)

fill_mask("Tona ya mapodisi Bheki Cele a re o tshwenngwa ke <mask> o o oketsegang wa bana ba bonnye ba dingwaga.")

/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:791: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


[{'score': 0.8257221579551697,
  'sequence': '<s>Tona ya mapodisi Bheki Cele a re o tshwenngwa ke mokgwa o o oketsegang wa bana ba bonnye ba dingwaga.</s>',
  'token': 1764,
  'token_str': 'Ġmokgwa'},
 {'score': 0.03127484396100044,
  'sequence': '<s>Tona ya mapodisi Bheki Cele a re o tshwenngwa ke molato o o oketsegang wa bana ba bonnye ba dingwaga.</s>',
  'token': 1312,
  'token_str': 'Ġmolato'},
 {'score': 0.01745794527232647,
  'sequence': '<s>Tona ya mapodisi Bheki Cele a re o tshwenngwa ke ngwaga o o oketsegang wa bana ba bonnye ba dingwaga.</s>',
  'token': 634,
  'token_str': 'Ġngwaga'},
 {'score': 0.011269859038293362,
  'sequence': '<s>Tona ya mapodisi Bheki Cele a re o tshwenngwa ke monna o o oketsegang wa bana ba bonnye ba dingwaga.</s>',
  'token': 746,
  'token_str': 'Ġmonna'},
 {'score': 0.006323557812720537,
  'sequence': '<s>Tona ya mapodisi Bheki Cele a re o tshwenngwa ke molao o o oketsegang wa bana ba bonnye ba dingwaga.</s>',
  'token': 833,
  'token_str': 'Ġmolao

## save model on huggingface site

In [ ]:
from transformers import TFRobertaForMaskedLM

tf_model = TFRobertaForMaskedLM.from_pretrained("./TswanaBert", from_pt=True)
tf_model.save_pretrained("./TswanaBert")

All PyTorch model weights were used when initializing TFRobertaForMaskedLM.

Some weights or buffers of the PyTorch model TFRobertaForMaskedLM were not initialized from the TF 2.0 model and are newly initialized: ['lm_head.decoder.bias', 'lm_head.decoder.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Data sources

1. The largest portion of this dataset (10k)  sentences of text, comes from the [Leipzig Corpora Collection](https://wortschatz.uni-leipzig.de/en/download)


2. I Then added SABC news headlines collected by Marivate Vukosi, & Sefara Tshephisho, (2020)  that is generously made available on [zenoodo](http://doi.org/10.5281/zenodo.3668495 ). This added 185 tswana sentences to my corpus. 


3. I went on to add 300 more sentences by scrapping following sites news sites and blogs that mostly based in Botswana. I actively continue to expand the dataset.

* http://setswana.blogspot.com/
* https://omniglot.com/writing/tswana.php
* http://www.dailynews.gov.bw/
* http://www.mmegi.bw/index.php
* https://tsena.co.bw
* http://www.botswana.co.za/Cultural_Issues-travel/botswana-country-guide-en-route.html
* https://www.poemhunter.com/poem/2013-setswana/
https://www.poemhunter.com/poem/ngwana-wa-mosetsana/
 




In [ ]:
!transformers-cli login

2020-07-01 21:46:45.170336: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1

        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: MoseliMotsoehli
Password: 
Login successful
Your token: uILpqJCPboRgIqtOZcthQNyjnJUTpVVobIurrCjjfTrkEBXuGvmIXCUjPbnokBIFNfffBpdBoSazxNEvYyjBzGeKUURKzwASyXkDOZcKunAiMnMwZKSFKjvJbWtACXCy 

Your token has been saved to /root/.huggingface/token


In [ ]:
!transformers-cli upload ./TswanaBert/

2020-07-01 21:48:29.956851: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
About to upload file /content/TswanaBert/vocab.json to S3 under filename TswanaBert/vocab.json and namespace MoseliMotsoehli
About to upload file /content/TswanaBert/special_tokens_map.json to S3 under filename TswanaBert/special_tokens_map.json and namespace MoseliMotsoehli
About to upload file /content/TswanaBert/tokenizer_config.json to S3 under filename TswanaBert/tokenizer_config.json and namespace MoseliMotsoehli
About to upload file /content/TswanaBert/training_args.bin to S3 under filename TswanaBert/training_args.bin and namespace MoseliMotsoehli
About to upload file /content/TswanaBert/pytorch_model.bin to S3 under filename TswanaBert/pytorch_model.bin and namespace MoseliMotsoehli
About to upload file /content/TswanaBert/config.json to S3 under filename TswanaBert/config.json and namespace MoseliMotsoehli
About to upload file /cont

## Use model from hugging face

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("MoseliMotsoehli/TswanaBert")
model = AutoModelWithLMHead.from_pretrained("MoseliMotsoehli/TswanaBert")

unmasker = pipeline('fill-mask', model=model, tokenizer=tokenizer)

/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:791: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
unmasker("Leina la <mask> ke.")

[{'score': 0.5053399205207825,
  'sequence': '<s>Leina la me ke.</s>',
  'token': 370,
  'token_str': 'Ġme'},
 {'score': 0.3169836699962616,
  'sequence': '<s>Leina la ntlha ke.</s>',
  'token': 647,
  'token_str': 'Ġntlha'},
 {'score': 0.025851018726825714,
  'sequence': '<s>Leina la gago ke.</s>',
  'token': 823,
  'token_str': 'Ġgago'},
 {'score': 0.020124521106481552,
  'sequence': '<s>Leina la gagwe ke.</s>',
  'token': 409,
  'token_str': 'Ġgagwe'},
 {'score': 0.00885735172778368,
  'sequence': '<s>Leina la ga ke.</s>',
  'token': 297,
  'token_str': 'Ġga'}]